In [1]:
import numpy as np
import pandas as pd
import polars as pl

from collections import deque, defaultdict
import ast
import os
import glob
from pathlib import Path
from scipy.stats import mode
from modeling_parameters.config import (
    MATPLOTLIBRC_PATH,
    COORDINATES_PATH,
    CENTRAL_STATIONS,
)

import matplotlib as mpl
mpl.rc_file(MATPLOTLIBRC_PATH)
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import seaborn as sns

from modeling_parameters.reconstruction.geant.functions import(
     compute_r,
     rho_model,
     filter_clusters,
)

from functions import(
    ast_df,
    ast_modeling_df,
    is_central_stations,
)

from config import(
    converters,
    columns_to_eval,
)


In [2]:
coordinates_df = pd.read_csv(COORDINATES_PATH)

X_det = np.array(coordinates_df['X'])
Y_det = np.array(coordinates_df['Y'])
Z_det = np.array(coordinates_df['Z'])

In [3]:
geant_dir = '../../modeling_parameters/read/geant/output/spe27/'
geant_1_file = 'spe27p_100.parquet'
geant_2_file = 'spe27p1416_150k.parquet'


geant_1_filepath  = os.path.join(geant_dir, geant_1_file)
geant_2_filepath  = os.path.join(geant_dir, geant_2_file)


In [4]:
df = pl.read_parquet(geant_1_filepath)

In [5]:
df.head(1)

N_event,NRUN,NEVENT,PART0,E0,Teta,Fi,XAxisShift,YAxisShift,H1INT,NGAM,NEL,NHADR,NMU,NeNKGlong,sNKGlong,NVD_edep,NVD_npe,MuBundle,MuTrackLenNVD,nMuNVD,eMuNVD,eMuNVD1,muDCR,muSM,nSM,muDCRw,muSMw,nSMw,muDCR0,muSM0,nSM0,muDCRw0,muSMw0,nSMw0,AmplKSM,hSM,nTrackSMX,nTrackSMY,nTrackSM,hSM0,nTrackSMX0,nTrackSMY0,nTrackSM0,EdepCntSCT,EdepDetNE,TimDetNE,EdepStNE,TimStNE,marker
f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,list[list[list[list[f64]]]],list[f64],list[f64],list[f64],list[f64],list[f64],list[f64],list[f64],list[f64],list[list[list[f64]]],list[list[list[f64]]],list[list[list[list[f64]]]],list[list[f64]],list[list[list[f64]]],f64
0.0,1000.0,1.0,14.0,1.024144e6,37.367474,47.313324,13.572407,37.022316,2.2006e6,139558.0,18742.0,815.0,10006.0,14672.599609,1.444716,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"[[[[0.0, 0.0, … 0.0], [0.0, 0.0, … 0.0], … [0.0, 0.0, … 0.0]], [[0.0, 0.0, … 0.0], [0.0, 0.0, … 0.0], … [0.0, 0.0, … 0.0]], … [[0.0, 0.0, … 0.0], [0.0, 0.0, … 0.0], … [0.0, 0.0, … 0.0]]], [[[0.0, 0.0, … 0.0], [0.0, 0.0, … 0.0], … [0.0, 0.0, … 0.0]], [[0.0, 0.0, … 0.0], [0.0, 0.0, … 0.0], … [0.0, 0.0, … 0.0]], … [[0.0, 0.0, … 0.0], [0.0, 0.0, … 0.0], … [0.0, 0.0, … 0.0]]], … [[[0.0, 0.0, … 0.0], [0.0, 0.0, … 0.0], … [0.0, 0.0, … 0.0]], [[0.0, 0.0, … 0.0], [0.0, 0.0, … 0.0], … [0.0, 0.0, … 0.0]], … [[0.0, 0.0, … 0.0], [0.0, 0.0, … 0.0], … [0.0, 0.0, … 0.0]]]]","[0.0, 0.0, … 0.0]","[0.0, 0.0, … 0.0]","[0.0, 0.0, … 0.0]","[0.0, 0.0, … 0.0]","[0.0, 0.0, … 0.0]","[0.0, 0.0, … 0.0]","[0.0, 0.0, … 0.0]","[0.0, 0.0, … 0.0]","[[[0.0, 0.0], [0.0, 0.0], … [0.0, 0.0]], [[0.0, 0.0], [0.0, 0.0], … [0.0, 0.0]], … [[0.0, 0.0], [0.0, 0.0], … [0.0, 0.0]]]","[[[0.0, 0.0, … 0.0], [10.222194, 1.102166, … 0.0], … [0.0, 0.0, … 12.019694]], [[0.0, 0.0, … 21.797497], [28.810249, 1.386011, … 16.11937], … [8.516803, 0.0, … 0.0]], … [[0.0, 0.607349, … 0.0], [11.273481, 0.0, … 0.024522], … [0.0, 0.0, … 0.0]]]","[[[[0.0, -1.0, … 0.0], [0.0, -1.0, … 0.0], … [0.0, -1.0, … 0.0]], [[91830.773438, 91830.726562, … 91830.71875], [91847.273438, -1.0, … 91847.234375], … [0.0, -1.0, … 0.0]], … [[0.0, -1.0, … 0.0], [0.0, -1.0, … 0.0], … [91830.789062, 91830.710938, … 91830.710938]]], [[[0.0, -1.0, … 0.0], [0.0, -1.0, … 0.0], … [91806.085938, 91806.085938, … 91777.171875]], [[91788.414062, 91781.90625, … 91782.5], [91788.851562, -1.0, … 91785.203125], … [91784.054688, 91783.734375, … 91783.804688]], … [[91779.9375, 91779.9375, … 91779.9375], [0.0, -1.0, … 0.0], … [0.0, -1.0, … 0.0]]], … [[[0.0, -1.0, … 0.0], [91910.6875, -1.0, … 91910.6875], … [0.0, -1.0, … 0.0]], [[91878.546875, 91878.492188, … 91878.476562], [0.0, -1.0, … 0.0], … [91881.507812, -1.0, … 91881.507812]], … [[0.0, -1.0, … 0.0], [0.0, -1.0, … 0.0], … [0.0, -1.0, … 0.0]]]]","[[0.101911, 11.324359, … 23.782166], [21.877735, 57.119301, … 8.516803], … [0.624726, 11.298003, … 0.0]]","[[[91820.75, -1.0, … 91820.75], [91847.273438, 91830.726562, … 91833.71875], … [91832.429688, 91830.710938, … 91830.71875]], [[91806.085938, 91806.085938, … 91777.960938], [91788.851562, 91781.867188, … 91782.921875], … [91779.9375, 91779.9375, … 91779.9375]], … [[91910.6875, -1.0, … 91895.914062], [91881.507812, 91878.492188, … 91878.726562], … [0.0, -1.0, … 0.0]]]",-1.0


In [6]:
df_filtered = df.filter(pl.col("Teta") >= 40)

In [7]:
def prepare_event(E_1_event):
    e_list = []
    for i_cluster, cluster in enumerate(E_1_event):
        for j_station, value in enumerate(cluster):
            e_list.append({'cluster': i_cluster+1, 'station': j_station+1, 'E': value})
    return pd.DataFrame(e_list)

In [8]:
df_pd = df_filtered.to_pandas()

r_modeling = []
rho_modeling = []
final_rows = []


for j_event in range(len(df_pd)):
    theta_g = df_pd.at[j_event, 'Teta']
    phi_g = df_pd.at[j_event, 'Fi']
    X0_g = df_pd.at[j_event, 'XAxisShift']
    Y0_g = df_pd.at[j_event, 'YAxisShift']
    Z0_g = 0

    r_geant = compute_r(X0_g, Y0_g, Z0_g, theta_g, phi_g, X_det, Y_det, Z_det)

    E_1_event = df_pd.at[j_event, 'EdepStNE']
    E_stations = prepare_event(E_1_event)

    worked_clusters, worked_stations, mask, rho_geant = filter_clusters(E_stations, threshold=0)

    central_subset = E_stations[E_stations['station'].isin(CENTRAL_STATIONS)]
    top2 = central_subset.nlargest(2, 'E')

    if len(top2) == 2 and np.any(mask): 
        r_vals = r_geant[mask]
        rho_vals = rho_geant  

        if r_vals.size > 0 and rho_vals.size > 0:
            row = df_pd.iloc[j_event].copy()
            row['r_vals'] = r_vals.tolist()
            row['rho_vals'] = rho_vals.tolist()
            final_rows.append(row)


final_df = pd.DataFrame(final_rows)
final_pl_df = pl.from_pandas(final_df)
final_pl_df.write_parquet("geant_data/spe/spe27_theta_ge_40.parquet")